# COS711 Assignment 3
## Student u20798441
## Predicting Pollution Levels Based on Time Series Weather Data

## Data Import

In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [2]:
# import and inspect the provided training data.
train=pd.read_csv("Train.csv")

In [3]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [4]:
print(train[['temp']].values[34])

['20.525,19.68333333,19.20833333,19.70833333,20.08333333,20.275,20.69166667,22.65833333,23.24166667,23.58333333,24.59166667,24.625,23.3,18.73333333,18.85,18.75,18.375,18.51666667,18.90833333,19.06666667,19.19166667,19.25833333,19.15,19.04166667,19.075,18.5,17.95833333,17.94166667,18.26666667,19.86666667,22.59166667,24.18333333,25.25833333,26.16666667,24.05,24.125,24.50833333,23.225,22.075,21.575,21.175,20.83333333,20.65833333,20.75,20.425,20.19166667,19.94166667,19.84166667,19.825,19.78333333,19.73333333,19.81666667,20.31666667,21.85,22.28333333,23.0,24.025,25.06666667,23.2,20.075,19.95833333,20.69166667,20.73333333,20.58333333,20.325,19.73333333,19.4,19.53333333,19.54166667,19.29166667,19.40833333,19.70833333,19.56666667,19.49166667,19.50833333,19.44166667,20.41666667,22.34166667,23.25833333,24.775,25.10833333,25.83333333,26.05833333,25.64166667,24.9,23.35,21.96666667,20.73333333,19.76666667,19.375,19.09166667,18.2,18.71666667,18.90833333,19.09166667,19.14166667,19.16666667,19.2333333

## Process Missing Values

In [6]:
# A function to replace missing values with averages of the closest values.
def nan_to_value(vals):
    last_val = ""
    next_val = ""
    new_val = ""
    
    for i in range(len(vals)):
        if vals[i] != "nan":
            last_val = vals[i]
        else:
            for j in range(i, len(vals)):
                if vals[j] != "nan":
                    next_val = vals[j]
                    break
            
            if next_val == "":
                new_val = last_val
            elif last_val == "":
                new_val = next_val
            else:
                new_val = str((float(last_val) + float(next_val)) / 2)
            vals[i] = new_val
            last_val = new_val
            next_val = ""
            new_val = ""
            
    return vals

In [7]:
# Replace missing values.
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
sep = ','

for feature in features : 
    for q in range(len(train[feature])):
        train.at[q, feature] = sep.join(nan_to_value(train[feature].values[q].split(",")))

In [8]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"26.90909091,26.90909091,26.90909091,26.9090909...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.5108181820000001,0.5108181820000001,0.510818...","272.90275160000004,272.90275160000004,272.9027...","0.800909091,0.800909091,0.800909091,0.80090909...","87.77727273,87.77727273,87.77727273,87.7772727...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","328.68291380000005,328.68291380000005,328.6829...","1.0325,1.0325,1.0325,1.0325,1.0325,1.0325,1.03...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


## Structuring Data

In [9]:
# Convert all values to floats to process with neural network.
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ float(X) for X in x.split(",")])

In [10]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"[26.90909091, 26.90909091, 26.90909091, 26.909...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5108181820000001, 0.5108181820000001, 0.510...","[272.90275160000004, 272.90275160000004, 272.9...","[0.800909091, 0.800909091, 0.800909091, 0.8009...","[87.77727273, 87.77727273, 87.77727273, 87.777...",45.126304
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[328.68291380000005, 328.68291380000005, 328.6...","[1.0325, 1.0325, 1.0325, 1.0325, 1.0325, 1.032...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304
3,ID_train_100,A,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...",53.850238
4,ID_train_1000,A,"[21.875, 21.575, 21.525, 21.43333333, 20.50833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8564166670000001, 0.874916667, 0.879833333,...","[21.83997432, 17.05405341, 89.26406044, 123.58...","[0.1975, 0.244166667, 0.411666667, 0.56, 0.577...","[88.55666667, 88.64083333, 88.65833333, 88.647...",177.418750


## Feature Engineering

In [11]:
# Create a copy of the dataframe becauase different preparation steps are used for the different models.
dim_train = train.copy()

In [12]:
# Expand the dataset for the FFN with some statistics.
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  

In [13]:
for col_name in tqdm(features):
    train=aggregate_features(train,col_name)

100%|██████████| 6/6 [00:11<00:00,  1.86s/it]


In [14]:
train.columns.values

array(['ID', 'location', 'temp', 'precip', 'rel_humidity', 'wind_dir',
       'wind_spd', 'atmos_press', 'target', 'max_temp', 'min_temp',
       'mean_temp', 'std_temp', 'var_temp', 'median_temp', 'ptp_temp',
       'max_precip', 'min_precip', 'mean_precip', 'std_precip',
       'var_precip', 'median_precip', 'ptp_precip', 'max_rel_humidity',
       'min_rel_humidity', 'mean_rel_humidity', 'std_rel_humidity',
       'var_rel_humidity', 'median_rel_humidity', 'ptp_rel_humidity',
       'max_wind_dir', 'min_wind_dir', 'mean_wind_dir', 'std_wind_dir',
       'var_wind_dir', 'median_wind_dir', 'ptp_wind_dir', 'max_wind_spd',
       'min_wind_spd', 'mean_wind_spd', 'std_wind_spd', 'var_wind_spd',
       'median_wind_spd', 'ptp_wind_spd', 'max_atmos_press',
       'min_atmos_press', 'mean_atmos_press', 'std_atmos_press',
       'var_atmos_press', 'median_atmos_press', 'ptp_atmos_press'],
      dtype=object)

In [15]:
train.shape

(15539, 51)

In [16]:
dim_train.shape

(15539, 9)

In [17]:
# flatten the arraiys in the attributes by making the series attributes. 

for x in range(121):
    train["newtemp"+ str(x)] = train.temp.str[x]
    train["newprecip"+ str(x)] = train.precip.str[x]
    train["newrel_humidity"+ str(x)] = train.rel_humidity.str[x]
    train["newwind_dir"+ str(x)] = train.wind_dir.str[x]
    train["windspeed"+ str(x)] = train.wind_spd.str[x]
    train["atmospherepressure"+ str(x)] = train.atmos_press.str[x]
    
    dim_train["newtemp"+ str(x)] = dim_train.temp.str[x]
    dim_train["newprecip"+ str(x)] = dim_train.precip.str[x]
    dim_train["newrel_humidity"+ str(x)] = dim_train.rel_humidity.str[x]
    dim_train["newwind_dir"+ str(x)] = dim_train.wind_dir.str[x]
    dim_train["windspeed"+ str(x)] = dim_train.wind_spd.str[x]
    dim_train["atmospherepressure"+ str(x)] = dim_train.atmos_press.str[x]

In [17]:
train.shape

(15539, 777)

In [18]:
# remove the original multi-dimentional attributes as we do not want duplicaiton in our data.
train.drop(features,1,inplace=True)
dim_train.drop(features,1,inplace=True)

In [19]:
train.shape

(15539, 771)

In [20]:
train.head()

,ID,location,target,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,...,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120
0,ID_train_0,C,45.126304,27.208333,19.275000,26.299562,1.824256,3.327909,26.909091,7.933333,...,0.881500,329.178292,0.673333,87.750833,19.583333,0.000,0.874583,325.478885,0.855000,87.768333
1,ID_train_1,D,79.131702,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,...,0.649250,73.528733,1.475833,90.439167,21.450000,0.561,0.797333,296.967254,1.019167,90.529167
2,ID_train_10,A,32.661304,31.841667,18.458333,24.100025,3.763149,14.161292,23.300000,13.383333,...,0.459583,209.603600,1.481667,88.475833,30.091667,0.000,0.437917,250.105258,1.746667,88.415833
3,ID_train_100,A,53.850238,27.491667,16.941667,20.845273,3.080140,9.487259,19.541667,10.550000,...,0.766417,335.844256,1.367500,88.490833,24.641667,0.000,0.719667,331.118658,1.852500,88.465833
4,ID_train_1000,A,177.418750,28.750000,17.525000,21.042443,1.720495,2.960103,20.616667,11.225000,...,0.831417,62.391650,0.735000,88.465833,20.616667,0.000,0.831417,62.391650,0.735000,88.465833


In [21]:
# isolate the targets that we want to achieve.
targets = train['target']

In [22]:
# remove unwanted columns from both data frames.
train.drop('target',1,inplace=True)
dim_train.drop('target',1,inplace=True)

In [23]:
train.head()

,ID,location,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,...,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120
0,ID_train_0,C,27.208333,19.275000,26.299562,1.824256,3.327909,26.909091,7.933333,0.000,...,0.881500,329.178292,0.673333,87.750833,19.583333,0.000,0.874583,325.478885,0.855000,87.768333
1,ID_train_1,D,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,...,0.649250,73.528733,1.475833,90.439167,21.450000,0.561,0.797333,296.967254,1.019167,90.529167
2,ID_train_10,A,31.841667,18.458333,24.100025,3.763149,14.161292,23.300000,13.383333,7.804,...,0.459583,209.603600,1.481667,88.475833,30.091667,0.000,0.437917,250.105258,1.746667,88.415833
3,ID_train_100,A,27.491667,16.941667,20.845273,3.080140,9.487259,19.541667,10.550000,25.787,...,0.766417,335.844256,1.367500,88.490833,24.641667,0.000,0.719667,331.118658,1.852500,88.465833
4,ID_train_1000,A,28.750000,17.525000,21.042443,1.720495,2.960103,20.616667,11.225000,0.136,...,0.831417,62.391650,0.735000,88.465833,20.616667,0.000,0.831417,62.391650,0.735000,88.465833


In [24]:
dim_train.head()

,ID,location,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,...,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120
0,ID_train_0,C,26.909091,0.000,0.510818,272.902752,0.800909,87.777273,26.909091,0.0,...,0.881500,329.178292,0.673333,87.750833,19.583333,0.000,0.874583,325.478885,0.855000,87.768333
1,ID_train_1,D,22.533333,0.102,0.744583,281.664310,2.377500,90.320000,21.716667,0.0,...,0.649250,73.528733,1.475833,90.439167,21.450000,0.561,0.797333,296.967254,1.019167,90.529167
2,ID_train_10,A,28.975000,0.000,0.573333,328.682914,1.032500,88.551667,27.950000,0.0,...,0.459583,209.603600,1.481667,88.475833,30.091667,0.000,0.437917,250.105258,1.746667,88.415833
3,ID_train_100,A,22.966667,0.000,0.843083,300.085057,1.446667,88.615000,24.266667,0.0,...,0.766417,335.844256,1.367500,88.490833,24.641667,0.000,0.719667,331.118658,1.852500,88.465833
4,ID_train_1000,A,21.875000,0.000,0.856417,21.839974,0.197500,88.556667,21.575000,0.0,...,0.831417,62.391650,0.735000,88.465833,20.616667,0.000,0.831417,62.391650,0.735000,88.465833


In [25]:
# remove unwanted columns from both data frames.
train.drop('ID',1,inplace=True)
train.drop('location',1,inplace=True)

dim_train.drop('ID',1,inplace=True)
dim_train.drop('location',1,inplace=True)

In [26]:
dim_train.columns.values

array(['newtemp0', 'newprecip0', 'newrel_humidity0', 'newwind_dir0',
       'windspeed0', 'atmospherepressure0', 'newtemp1', 'newprecip1',
       'newrel_humidity1', 'newwind_dir1', 'windspeed1',
       'atmospherepressure1', 'newtemp2', 'newprecip2',
       'newrel_humidity2', 'newwind_dir2', 'windspeed2',
       'atmospherepressure2', 'newtemp3', 'newprecip3',
       'newrel_humidity3', 'newwind_dir3', 'windspeed3',
       'atmospherepressure3', 'newtemp4', 'newprecip4',
       'newrel_humidity4', 'newwind_dir4', 'windspeed4',
       'atmospherepressure4', 'newtemp5', 'newprecip5',
       'newrel_humidity5', 'newwind_dir5', 'windspeed5',
       'atmospherepressure5', 'newtemp6', 'newprecip6',
       'newrel_humidity6', 'newwind_dir6', 'windspeed6',
       'atmospherepressure6', 'newtemp7', 'newprecip7',
       'newrel_humidity7', 'newwind_dir7', 'windspeed7',
       'atmospherepressure7', 'newtemp8', 'newprecip8',
       'newrel_humidity8', 'newwind_dir8', 'windspeed8',
       'atmos

In [27]:
dim_train.head()

,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,...,newrel_humidity119,newwind_dir119,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120
0,26.909091,0.000,0.510818,272.902752,0.800909,87.777273,26.909091,0.0,0.510818,272.902752,...,0.881500,329.178292,0.673333,87.750833,19.583333,0.000,0.874583,325.478885,0.855000,87.768333
1,22.533333,0.102,0.744583,281.664310,2.377500,90.320000,21.716667,0.0,0.808083,89.156293,...,0.649250,73.528733,1.475833,90.439167,21.450000,0.561,0.797333,296.967254,1.019167,90.529167
2,28.975000,0.000,0.573333,328.682914,1.032500,88.551667,27.950000,0.0,0.597167,328.682914,...,0.459583,209.603600,1.481667,88.475833,30.091667,0.000,0.437917,250.105258,1.746667,88.415833
3,22.966667,0.000,0.843083,300.085057,1.446667,88.615000,24.266667,0.0,0.790250,293.676960,...,0.766417,335.844256,1.367500,88.490833,24.641667,0.000,0.719667,331.118658,1.852500,88.465833
4,21.875000,0.000,0.856417,21.839974,0.197500,88.556667,21.575000,0.0,0.874917,17.054053,...,0.831417,62.391650,0.735000,88.465833,20.616667,0.000,0.831417,62.391650,0.735000,88.465833


## Regularisation

In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
#Scalar for FFN
scaler = StandardScaler().fit(train)

# Scale the FFN network
train_scaled = scaler.transform(train)

In [30]:
# scalar for LSTM and CNN
dim_scaler = StandardScaler().fit(dim_train)

# Scale the LSTM & CNN networks
dim_train_scaled = dim_scaler.transform(dim_train)

In [31]:
train_scaled.shape

(15539, 768)

In [33]:
dim_train_scaled.shape

(15539, 726)

In [35]:
train_scaled

array([[-1.08572256,  1.60631863,  2.56170199, ...,  1.15863551,
        -0.26812093, -1.16812299],
       [ 1.88827143,  0.39717057,  1.40031376, ...,  0.90088391,
        -0.07539475,  1.36874035],
       [ 1.06452537,  0.84182501,  0.9853252 , ...,  0.47724086,
         0.77866595, -0.57315082],
       ...,
       [ 2.11644521, -0.57014787,  1.39179941, ..., -1.1470289 ,
         1.58674629,  1.40472965],
       [ 0.62751455, -1.56086918,  0.53589785, ..., -0.41256681,
         0.23766301,  1.18266797],
       [ 0.41867752,  1.38009092,  0.92267128, ...,  1.19254043,
        -0.28377382, -1.08848707]])

In [74]:
# Reshape for the LSTM network training
dim_train_shaped = dim_train_scaled.reshape((dim_train_scaled.shape[0], 121, 6))

In [75]:
dim_train_shaped.shape

(15539, 121, 6)

## Train - Test Split

In [42]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split for FFN
X_train, X_eval, y_train, y_eval = train_test_split(train_scaled, targets, test_size=0.20, random_state=40)

In [76]:
# Split for LSTM
X_train_dim, X_eval_dim, y_train_dim, y_eval_dim = train_test_split(dim_train_shaped, targets, test_size=0.20, random_state=40)

In [77]:
# Inspect the data
print(X_train.shape)
print(X_train_dim.shape)

(12431, 768)
(12431, 121, 6)


## FFN

In [45]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [46]:
# pruning setup for FFN training
from tensorflow_model_optimization.sparsity import keras as sparsity

batch_size = 1

epochs = 12
num_train_samples = X_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=20000,
                                                   end_step=end_step,
                                                   frequency=1000)
}

End step: 149172


In [47]:
# pruning setup for FFN training
callbacks = [
    sparsity.UpdatePruningStep()
]

In [62]:
# Initialize the constructor
model = Sequential()

# Input layer
model.add(Dense(768, activation='relu', input_dim=768))

# Hidden Layers
model.add(sparsity.prune_low_magnitude(Dense(1024, activation='relu')))
model.add(sparsity.prune_low_magnitude(Dense(512, activation='relu')))
model.add(sparsity.prune_low_magnitude(Dense(128, activation='relu')))
model.add(sparsity.prune_low_magnitude(Dense(32, activation='relu')))

# Output layer 
model.add(Dense(1))

In [63]:
model.compile(loss='mse',
              optimizer='Adam',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])
                   
model.fit(X_train, y_train, 
          epochs=64,
          verbose=1,
          callbacks=callbacks)

Train on 12431 samples
Epoch 1/64
12431/12431 [==============================] - 4s 302us/sample - loss: 1285.1900 - root_mean_squared_error: 35.8495
Epoch 2/64
12431/12431 [==============================] - 3s 207us/sample - loss: 990.9822 - root_mean_squared_error: 31.4799
Epoch 3/64
12431/12431 [==============================] - 3s 210us/sample - loss: 856.2088 - root_mean_squared_error: 29.2610
Epoch 4/64
12431/12431 [==============================] - 3s 203us/sample - loss: 763.9521 - root_mean_squared_error: 27.6397
Epoch 5/64
12431/12431 [==============================] - 3s 205us/sample - loss: 675.9504 - root_mean_squared_error: 25.9990
Epoch 6/64
12431/12431 [==============================] - 3s 204us/sample - loss: 573.0048 - root_mean_squared_error: 23.9375
Epoch 7/64
12431/12431 [==============================] - 3s 204us/sample - loss: 500.9168 - root_mean_squared_error: 22.3812
Epoch 8/64
12431/12431 [==============================] - 3s 204us/sample - loss: 426.4908 - r

In [64]:
# Evaluate FFN model directly after 64 epochs of training with pruning
eval_results = model.evaluate(X_eval, y_eval, verbose=1)

3108/3108 [==============================] - 0s 93us/sample - loss: 895.5286 - root_mean_squared_error: 29.9254


In [65]:
# inspect model to see weiths before proning removed.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 768)               590592    
_________________________________________________________________
prune_low_magnitude_dense_7  (None, 1024)              1573890   
_________________________________________________________________
prune_low_magnitude_dense_8  (None, 512)               1049090   
_________________________________________________________________
prune_low_magnitude_dense_9  (None, 128)               131202    
_________________________________________________________________
prune_low_magnitude_dense_10 (None, 32)                8226      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 3,353,033
Trainable params: 1,972,673
Non-trainable params: 1,380,360
____________________________________

In [66]:
# Remove pruning from model and observe the reduced number of parameters.
ffn_model = sparsity.strip_pruning(model)
ffn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 768)               590592    
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              787456    
_________________________________________________________________
dense_8 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 1,972,673
Trainable params: 1,972,673
Non-trainable params: 0
____________________________________________

In [67]:
# Calculate the amount of shrinking that happened with pruning.
for i, w in enumerate(ffn_model.get_weights()):
    print("{} -- Total:{}, Zeros: {:.2f}%".format(ffn_model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100))

dense_6/kernel:0 -- Total:589824, Zeros: 0.00%
dense_6/bias:0 -- Total:768, Zeros: 0.00%
prune_low_magnitude_dense_7/kernel:0 -- Total:786432, Zeros: 50.00%
prune_low_magnitude_dense_7/bias:0 -- Total:1024, Zeros: 0.00%
prune_low_magnitude_dense_8/kernel:0 -- Total:524288, Zeros: 50.00%
prune_low_magnitude_dense_8/bias:0 -- Total:512, Zeros: 0.00%
prune_low_magnitude_dense_9/kernel:0 -- Total:65536, Zeros: 50.00%
prune_low_magnitude_dense_9/bias:0 -- Total:128, Zeros: 0.00%
prune_low_magnitude_dense_10/kernel:0 -- Total:4096, Zeros: 50.00%
prune_low_magnitude_dense_10/bias:0 -- Total:32, Zeros: 3.12%
dense_11/kernel:0 -- Total:32, Zeros: 0.00%
dense_11/bias:0 -- Total:1, Zeros: 0.00%


In [68]:
# Continue training FFN model after pruning has been removed.
ffn_model.compile(loss='mse',
              optimizer='Adam',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

ffn_model.fit(X_train, y_train, 
          epochs=192,
          verbose=1)

Train on 12431 samples
Epoch 1/192
12431/12431 [==============================] - 2s 130us/sample - loss: 41.5746 - root_mean_squared_error: 6.4478
Epoch 2/192
12431/12431 [==============================] - 1s 103us/sample - loss: 55.8942 - root_mean_squared_error: 7.4762
Epoch 3/192
12431/12431 [==============================] - 1s 103us/sample - loss: 45.9834 - root_mean_squared_error: 6.7811
Epoch 4/192
12431/12431 [==============================] - 1s 101us/sample - loss: 39.3231 - root_mean_squared_error: 6.2708
Epoch 5/192
12431/12431 [==============================] - 1s 102us/sample - loss: 39.5389 - root_mean_squared_error: 6.2880
Epoch 6/192
12431/12431 [==============================] - 1s 102us/sample - loss: 35.1438 - root_mean_squared_error: 5.9282
Epoch 7/192
12431/12431 [==============================] - 1s 101us/sample - loss: 36.2776 - root_mean_squared_error: 6.0231
Epoch 8/192
12431/12431 [==============================] - 1s 102us/sample - loss: 39.9009 - root_mean

12431/12431 [==============================] - 1s 103us/sample - loss: 19.1921 - root_mean_squared_error: 4.3809
Epoch 66/192
12431/12431 [==============================] - 1s 102us/sample - loss: 17.2593 - root_mean_squared_error: 4.1544
Epoch 67/192
12431/12431 [==============================] - 1s 101us/sample - loss: 14.7973 - root_mean_squared_error: 3.8467
Epoch 68/192
12431/12431 [==============================] - 1s 101us/sample - loss: 15.4422 - root_mean_squared_error: 3.9297
Epoch 69/192
12431/12431 [==============================] - 1s 101us/sample - loss: 13.9848 - root_mean_squared_error: 3.7396
Epoch 70/192
12431/12431 [==============================] - 1s 101us/sample - loss: 14.0544 - root_mean_squared_error: 3.7489
Epoch 71/192
12431/12431 [==============================] - 1s 101us/sample - loss: 14.9906 - root_mean_squared_error: 3.8718
Epoch 72/192
12431/12431 [==============================] - 1s 101us/sample - loss: 18.1684 - root_mean_squared_error: 4.2624
Epoch

12431/12431 [==============================] - 1s 100us/sample - loss: 9.8058 - root_mean_squared_error: 3.1314
Epoch 130/192
12431/12431 [==============================] - 1s 100us/sample - loss: 8.2911 - root_mean_squared_error: 2.8794
Epoch 131/192
12431/12431 [==============================] - 1s 100us/sample - loss: 7.4420 - root_mean_squared_error: 2.7280
Epoch 132/192
12431/12431 [==============================] - 1s 100us/sample - loss: 13.6977 - root_mean_squared_error: 3.7010
Epoch 133/192
12431/12431 [==============================] - 1s 100us/sample - loss: 10.9517 - root_mean_squared_error: 3.3093
Epoch 134/192
12431/12431 [==============================] - 1s 100us/sample - loss: 9.3365 - root_mean_squared_error: 3.0556
Epoch 135/192
12431/12431 [==============================] - 1s 101us/sample - loss: 7.2517 - root_mean_squared_error: 2.6929
Epoch 136/192
12431/12431 [==============================] - 1s 101us/sample - loss: 8.8868 - root_mean_squared_error: 2.9811
Epoc

In [69]:
# Evaluate model after more training. Observe that the RMSE increased slightly. On some of the runs, the model
# proved to be better after more training.
final_eval_results = ffn_model.evaluate(X_eval, y_eval, verbose=1)

3108/3108 [==============================] - 0s 78us/sample - loss: 902.2969 - root_mean_squared_error: 30.0383


## LSTM

In [49]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM#, CuDNNLSTM

import keras
from keras.layers import CuDNNLSTM
from keras import backend as K

In [78]:
X_train_dim.shape

(12431, 121, 6)

In [81]:
model2 = Sequential()

# Input layer
model2.add(LSTM(128, input_shape=(121,6), return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

# Hidden layers
model2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

model2.add(Flatten())

model2.add(Dense(1024, activation='relu'))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(32, activation='relu'))

# Output Layer
model2.add(Dense(1))

In [92]:
model2.compile(
    loss='mse',
    optimizer='Adam',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)


model2.fit(X_train_dim,
          y_train_dim,
          epochs=32)

Train on 12431 samples
Epoch 1/32
12431/12431 [==============================] - 120s 10ms/sample - loss: 693.7766 - root_mean_squared_error: 26.3396
Epoch 2/32
12431/12431 [==============================] - 117s 9ms/sample - loss: 669.3559 - root_mean_squared_error: 25.8719
Epoch 3/32
12431/12431 [==============================] - 117s 9ms/sample - loss: 694.7400 - root_mean_squared_error: 26.3579
Epoch 4/32
12431/12431 [==============================] - 117s 9ms/sample - loss: 658.4402 - root_mean_squared_error: 25.6601
Epoch 5/32
12431/12431 [==============================] - 117s 9ms/sample - loss: 674.2310 - root_mean_squared_error: 25.9660
Epoch 6/32
12431/12431 [==============================] - 116s 9ms/sample - loss: 652.1651 - root_mean_squared_error: 25.5375
Epoch 7/32
12431/12431 [==============================] - 114s 9ms/sample - loss: 653.8295 - root_mean_squared_error: 25.5701
Epoch 8/32
12431/12431 [==============================] - 114s 9ms/sample - loss: 661.4583 - r

In [93]:
# Evaluate model after more training.
eval_results = model2.evaluate(X_eval_dim, y_eval_dim, verbose=1)

3108/3108 [==============================] - 5s 2ms/sample - loss: 930.2679 - root_mean_squared_error: 30.5003


In [150]:
# This model was thrained in 4 phases, acheiving a better RMSE after every phase.

## CNN

In [102]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [126]:
# Transform the data into the required form for a CNN layer.
dim_train_shaped_conv = dim_train_scaled.reshape((dim_train_scaled.shape[0], 121, 6, 1))

In [127]:
dim_train_shaped_conv.shape

(15539, 121, 6, 1)

In [128]:
X_train_dim_conv, X_eval_dim_conv, y_train_dim_conv, y_eval_dim_conv = train_test_split(dim_train_shaped_conv, targets, test_size=0.20, random_state=40)

In [143]:
model3 = Sequential()

# Input layer
model3.add(Conv2D(filters=64, kernel_size=(1, 5), strides=(1, 2), activation="relu", input_shape=(121,6,1)))

# Hidden layers
model3.add(MaxPooling2D(pool_size=(2, 1)))

model3.add(Conv2D(filters=128, kernel_size=(5, 1), activation="relu"))
model3.add(MaxPooling2D(pool_size=(2, 1)))

model3.add(Flatten())

model3.add(Dense(1024, activation="relu"))
model3.add(Dense(256, activation="relu"))
model3.add(Dense(32, activation="relu"))

# Output Layer.
model3.add(Dense(1))

In [146]:
model3.compile(
    loss='mse',
    optimizer='Adam',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model3.fit(X_train_dim_conv,
          y_train_dim_conv,
          epochs=128)

Train on 12431 samples
Epoch 1/128
12431/12431 [==============================] - 2s 196us/sample - loss: 9.5968 - root_mean_squared_error: 3.0979
Epoch 2/128
12431/12431 [==============================] - 2s 164us/sample - loss: 9.7442 - root_mean_squared_error: 3.1216
Epoch 3/128
12431/12431 [==============================] - 2s 163us/sample - loss: 7.6903 - root_mean_squared_error: 2.7731
Epoch 4/128
12431/12431 [==============================] - 2s 169us/sample - loss: 8.1082 - root_mean_squared_error: 2.8475
Epoch 5/128
12431/12431 [==============================] - 2s 169us/sample - loss: 8.8288 - root_mean_squared_error: 2.9713
Epoch 6/128
12431/12431 [==============================] - 2s 166us/sample - loss: 8.3482 - root_mean_squared_error: 2.8893
Epoch 7/128
12431/12431 [==============================] - 2s 165us/sample - loss: 9.1989 - root_mean_squared_error: 3.0330
Epoch 8/128
12431/12431 [==============================] - 2s 166us/sample - loss: 9.1615 - root_mean_squared

12431/12431 [==============================] - 2s 162us/sample - loss: 6.0833 - root_mean_squared_error: 2.4664
Epoch 67/128
12431/12431 [==============================] - 2s 164us/sample - loss: 6.0622 - root_mean_squared_error: 2.4622
Epoch 68/128
12431/12431 [==============================] - 2s 164us/sample - loss: 5.9483 - root_mean_squared_error: 2.4389
Epoch 69/128
12431/12431 [==============================] - 2s 164us/sample - loss: 6.0860 - root_mean_squared_error: 2.4670
Epoch 70/128
12431/12431 [==============================] - 2s 167us/sample - loss: 5.4182 - root_mean_squared_error: 2.3277
Epoch 71/128
12431/12431 [==============================] - 2s 166us/sample - loss: 4.7478 - root_mean_squared_error: 2.1789
Epoch 72/128
12431/12431 [==============================] - 2s 167us/sample - loss: 4.9003 - root_mean_squared_error: 2.2137
Epoch 73/128
12431/12431 [==============================] - 2s 167us/sample - loss: 5.1653 - root_mean_squared_error: 2.2727
Epoch 74/128


In [147]:
# Evaluate model after more training.
eval_results3 = model3.evaluate(X_eval_dim_conv, y_eval_dim_conv, verbose=1)

3108/3108 [==============================] - 0s 108us/sample - loss: 813.8574 - root_mean_squared_error: 28.5282


In [151]:
# This model was thrained in 2 phases, acheiving a worse RMSE after the second phase as the model 
# already converged after the first phase.